In [0]:
spark.sql("USE CATALOG hive_metastore")

In [0]:
orders = spark.sql("select * from retail_test.orders")
orders.display()

In [0]:
%sql
select * from  retail_test.categories

In [0]:
%sql
select distinct year(orders.order_date) as orderyear from retail_test.orders

In [0]:
%sql
with base as
(
    select d.name as departmetnname,
    o.order_date,
    year(o.order_date) as year,
    case when month(o.order_date) in (1,2,3) then 'Q1'
    when month(o.order_date) in (4,5,6) then 'Q2'
    when month(o.order_date) in (7,8,9) then 'Q3'
    when month(o.order_date) in (10,11,12) then 'Q4'
    end as quarter,
    
    i.total as total
     from retail_test.orders o
     join retail_test.orderitems i on o.order_id = i.order_id
    join retail_test.products p on i.product_id = p.id    
    join retail_test.categories ca  on p.category_id = ca.id
    join retail_test.departments d on ca.department_d = d.id

),
--select * from base
--select departmetnname, concat(year,'-',quarter) as year_quarter, total as total from base
pivot_pre as
(select departmetnname, concat(year,'-',quarter) as year_quarter, total as total1 from base
)
 select * from pivot_pre pivot(sum(total1) as SALES for (year_quarter) in ('2013-Q3','2013-Q4','2014-Q1','2014-Q2','2014-Q3'))

In [0]:
df = spark.sql("select * from retail_test.orders where order_status in ('COMPLETE', 'CANCELED', 'CLOSED')")

In [0]:
df.createOrReplaceTempView("completed_orders")

In [0]:
%sql
select * from completed_orders

In [0]:
%sql
use retail_test;
show tables

In [0]:
df.createOrReplaceGlobalTempView("completed_orders_gbl")

In [0]:
%sql
select * from global_temp.completed_orders_gbl

# File types 

In [0]:
storage_account = "databricks180820251"


In [0]:
spark.sql("USE CATALOG hive_metastore")

In [0]:
df = spark.sql("select * from retail_test.orders")

In [0]:
df.display()

In [0]:
df.write.format("json").option("path",f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/formats/orders_json").mode("overwrite").save()

In [0]:
df = spark.read.json(f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/formats/orders_json")
df.display()

In [0]:
df.write.format("parquet").option("path",f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/formats/orders_parquet").mode("overwrite").save()

In [0]:
df.write.option("path",f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/formats/orders_delta").mode("overwrite").save()

In [0]:
df = spark.read.format("delta").load(f"abfss://silver@{storage_account}.dfs.core.windows.net/retail/formats/orders_delta")
df.display()